In [8]:
import pandas as pd
import numpy as np
import os

In [37]:
def load_grades(fname):
    # read excel file to identify header_row
    df = pd.read_excel(fname)
    
    # isolate instructor last name column
    first_col = df.iloc[:, 0]
    
    # if first row is not "Instructor Last Name", set header_row to the row that contains it.
    # otherwise, use first row as header_row
    try:
        header_row = first_col[first_col == "Instructor Last Name"].index[0]
    except IndexError as e:
        header_row = 0
        
    # re-read excel file with correct header_row
    grades = pd.read_excel(fname, header=header_row)
    
    # drop all rows that only have NA values
    grades = grades.dropna(how='all')
    return grades

def clean_grades(grade_df):
    # find appropriate column names
    course_number = grade_df.columns[5]
    subject = grade_df.columns[4]
    
    # drop any rows that have NA values in course number or subject columns
    grade_df = grade_df.dropna(how='any', subset=[course_number, subject])
    
    # if grade_df does not have "Totals" column
    if grade_df.shape[1] < 26:
    grade_df = grade_df.drop(columns=)
    return grade_df

In [54]:
def load_grade_files(data_dir):
    grades = {}
    # loop through all file names in data_dir
    for filename in os.listdir(data_dir):
        # don't read file if it is a temp excel swap file
        if filename.startswith("~"):
            continue
        print("Loading {}".format(filename))
        # construct file path for file
        file_path = os.path.join(data_dir, filename)
        # load excel file into dataframe
        grade_df = load_grades(file_path)
        # clean grade dataframe
        grade_df = clean_grades(grade_df)
        grades[filename] = grade_df
    return grades

grades = load_grade_files('excel_files')
len(grades)

Loading Grades_Fall_2015.xlsx
Loading Grades_Fall_2018.xlsx
Loading Grades_Fall_2014.xlsx
Loading Grades_Fall_2013.xlsx
Loading Grades_Spring_2016.xlsx
Loading Grades_Fall_2012.xlsx
Loading Grades_Spring_2015.xlsx
Loading Grades_Fall_2011.xlsx
Loading Grades_Spring_2014.xlsx
Loading Grades_Spring_2018.xlsx
Loading Grades_Spring_2013.xlsx
Loading Grades_Fall_2017.xlsx
Loading Grades_Fall_2016.xlsx
Loading Grades_Spring_2012.xlsx


14

In [59]:
def find_possible_column_names(grade_dict):

    max_columns = max([len(grade_df.columns) for filename, grade_df in grade_dict.items()])
    print(max_columns)



    for col in range(max_columns):
        print(set(df.columns[col] for fname, df in grade_dict.items() if col < len(df.columns)))
        print()

find_possible_column_names(grades)

26
{'Instructor Last Name'}

{'Instructor First', 'Instructor First Name'}

{'Instructor Middle Name'}

{'Instructor Email'}

{'Course Subject', 'Subject'}

{'Course Number'}

{'Section Number'}

{'Title', 'Course Title'}

{'Course GPA'}

{'A+'}

{'A'}

{'A-'}

{'B+'}

{'B'}

{'B-'}

{'C+'}

{'C'}

{'C-'}

{'D+'}

{'D'}

{'D-'}

{'F'}

{'OT', 'Other', 'othr'}

{'DR', 'Drop'}

{'W'}

{'Tot', 'Total'}



In [42]:
grades[0].shape[1]

25

In [43]:
grades[1].shape[1]

26

In [47]:
def numeric_conversion_errors(df, column_name):
    # create a temporary copy of the column
    column_copy = df[column_name].copy()
    # a list of error messages
    errors = []
    has_errors = True
    # loop while we encounter an error converting column_copy to numeric
    while has_errors:
        try:
            pd.to_numeric(column_copy)
        except Exception as e:
            # append error message
            errors.append("{}: {}".format(column_name, e))
            # find position of problematic value
            index = int(str(e).split()[-1])
            # replace with -1 to mark as "seen"
            column_copy.iloc[index] = -1
            continue
        # set has_errors to false if we don't encounter a problem during conversion
        has_errors = False
    return errors


def find_numeric_conversion_errors(df):
    # define columns that should be numeric
    numeric_columns = [5,6,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24]
    errors = []
    # loop through numeric columns and find all errors
    for col in numeric_columns:
        column_name = df.columns[col]
        print("Validating {}".format(column_name))
        col_errors = numeric_conversion_errors(df, column_name)
        if col_errors:
            print("\t" + "\n\t".join(col_errors))
            errors.extend(col_errors)
#     print(errors)
    return errors
        


In [64]:
def validate_grade_dfs(grade_dict):
    for fname, grade_df in grade_dict.items():
        print("###### Validating {}".format(fname))
        print(find_numeric_conversion_errors(grade_df))

def validate_grade_files(data_dir):
    grade_dict = load_grade_files(data_dir)
    validate_grade_dfs(grade_dict)

In [66]:
validate_grade_files('excel_files')

Loading Grades_Fall_2015.xlsx
Loading Grades_Fall_2018.xlsx
Loading Grades_Fall_2014.xlsx
Loading Grades_Fall_2013.xlsx
Loading Grades_Spring_2016.xlsx
Loading Grades_Fall_2012.xlsx
Loading Grades_Spring_2015.xlsx
Loading Grades_Fall_2011.xlsx
Loading Grades_Spring_2014.xlsx
Loading Grades_Spring_2018.xlsx
Loading Grades_Spring_2013.xlsx
Loading Grades_Fall_2017.xlsx
Loading Grades_Fall_2016.xlsx
Loading Grades_Spring_2012.xlsx
###### Validating Grades_Fall_2015.xlsx
Validating Course Number
Validating Section Number
Validating Course GPA
Validating A+
Validating A
Validating A-
Validating B+
Validating B
Validating B-
Validating C+
Validating C
Validating C-
Validating D+
Validating D
Validating D-
Validating F
Validating OT
Validating DR
Validating W
[]
###### Validating Grades_Fall_2018.xlsx
Validating Course Number
Validating Section Number
Validating Course GPA
Validating A+
Validating A
Validating A-
Validating B+
Validating B
Validating B-
Validating C+
Validating C
Validating C

In [163]:
validate_grades('S2018.xlsx')

Validating Course Number
Validating Section Number
Validating Course GPA
Validating A+
Validating A
Validating A-
Validating B+
Validating B
Validating B-
Validating C+
Validating C
Validating C-
Validating D+
Validating D
Validating D-
Validating F
Validating OT
Validating DR
Validating W
Validating Total


[]

In [164]:
validate_grades('F2017.xlsx')

Validating Course Number
Validating Section Number
Validating Course GPA
Validating A+
Validating A
Validating A-
Validating B+
Validating B
Validating B-
Validating C+
Validating C
Validating C-
Validating D+
Validating D
Validating D-
Validating F
Validating othr
Validating DR
Validating W


IndexError: index 25 is out of bounds for axis 0 with size 25